# Dynamic parameters

To facilitate experimenting around weighting, placeholder `Param`s can be utilised at query definition, which can be filled when running the query. 

In [1]:
%pip install superlinked==3.10.0

In [2]:
import pandas as pd

from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String, Integer
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace
from superlinked.framework.dsl.query.param import Param

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.space.number_space import NumberSpace, Mode
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

pd.set_option("display.max_colwidth", 100)

In [3]:
@schema
class Paragraph:
    id: IdField
    body: String
    like_count: Integer


paragraph = Paragraph()

body_space = TextSimilaritySpace(
    text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2"
)
like_space = NumberSpace(
    number=paragraph.like_count, min_value=0, max_value=100, mode=Mode.MAXIMUM
)
paragraph_index = Index([body_space, like_space])

In [4]:
source: InMemorySource = InMemorySource(paragraph)
executor = InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [5]:
source.put(
    [
        {
            "id": "paragraph-1",
            "body": "Glorious animals live in the wilderness.",
            "like_count": 75,
        },
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
            "like_count": 10,
        },
    ]
)

## Using dynamic Params in the query

When defining the query, placeholder `Param`s can be used to indicate actual weights and other parameters will be filled when running the query.

In [6]:
query = (
    Query(
        paragraph_index,
        weights={
            body_space: Param("body_space_weight"),
            like_space: Param("like_space_weight"),
        },
    )
    .find(paragraph)
    .similar(body_space.text, Param("query_text"))
)

In [7]:
body_based_result = app.query(
    query,
    query_text="How computation power changed the course of AI?",
    body_space_weight=1,
    like_space_weight=0,
)

pd.DataFrame([entry.stored_object for entry in body_based_result.entries])

,body,like_count,id
0,Growing computation power enables advancements in AI.,10,paragraph-2
1,Glorious animals live in the wilderness.,75,paragraph-1


In [8]:
like_based_result = app.query(
    query,
    query_text="How computation power changed the course of AI?",
    body_space_weight=0,
    like_space_weight=1,
)

pd.DataFrame([entry.stored_object for entry in like_based_result.entries])

,body,like_count,id
0,Glorious animals live in the wilderness.,75,paragraph-1
1,Growing computation power enables advancements in AI.,10,paragraph-2


This feature enables reusing the query for different setting by simply supplying it with alternate weightings.